In [1]:
import pandas as pd
import os

In [2]:

csv_files = [f for f in os.listdir("csvs") if f.endswith(".csv")]
csv_files

['insurance_filings_auto_pagination_AR.csv',
 'insurance_filings_auto_pagination_NJ.csv',
 'insurance_filings_auto_pagination_WI.csv',
 'insurance_filings_auto_pagination_UT.csv',
 'insurance_filings_auto_pagination_LA.csv',
 'insurance_filings_auto_pagination_ME.csv',
 'insurance_filings_auto_pagination_MS.csv',
 'insurance_filings_auto_pagination_NH.csv',
 'insurance_filings_auto_pagination_MD.csv',
 'insurance_filings_auto_pagination_CO.csv',
 'insurance_filings_auto_pagination_WY.csv',
 'insurance_filings_auto_pagination_OH.csv',
 'insurance_filings_auto_pagination_MA.csv',
 'insurance_filings_auto_pagination_NM.csv',
 'insurance_filings_auto_pagination_MT.csv',
 'insurance_filings_auto_pagination_OK.csv',
 'insurance_filings_auto_pagination_NY.csv',
 'insurance_filings_auto_pagination_SC.csv',
 'insurance_filings_auto_pagination_HI.csv',
 'insurance_filings_auto_pagination_IL.csv',
 'insurance_filings_auto_pagination_KS.csv',
 'insurance_filings_auto_pagination_IN.csv',
 'insuranc

In [ ]:
for file in csv_files:
    df = 

In [2]:
df = pd.read_csv("insurance_filings_manual_pagination.csv")

In [4]:
df['serf_num'] = df['SERFF Tracking Number'].str.split('-').str[1]


In [6]:
df['page_url'] = "https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId="+df.serf_num

In [7]:
df

,Company Name,NAIC Company Code,Insurance Product Name,Sub Type Of Insurance,Filing Type,Filing Status,SERFF Tracking Number,serf_num,page_url
0,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Grp Form 2015,H11G.003 Long Term,Form,Closed - Filed,CMPL-130322935,130322935,https://filingaccess.serff.com/sfa/search/fili...
1,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Large Small Grp Form 2015,H11G.003 Long Term,Form,Closed - Rejected,CMPL-130145349,130145349,https://filingaccess.serff.com/sfa/search/fili...
2,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Large Grp Rate 2015,H11G.003 Long Term,Rate,Closed - Rejected,CMPL-130144986,130144986,https://filingaccess.serff.com/sfa/search/fili...
3,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Large Grp Rate 2015,H11G.003 Long Term,Rate,Closed - Filed,CMPL-130324114,130324114,https://filingaccess.serff.com/sfa/search/fili...
4,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Small Grp Rate 2015,H11G.003 Long Term,Rate,Closed - Rejected,CMPL-130145065,130145065,https://filingaccess.serff.com/sfa/search/fili...
...,...,...,...,...,...,...,...,...,...
11415,Golden Rule Insurance Company,62286,CO Rates,H15I.001 Health - Hospital/Surgical/Medical Ex...,Rate,Closed - Filed,AMMS-126335346,126335346,https://filingaccess.serff.com/sfa/search/fili...
11416,Golden Rule Insurance Company,62286,Colorado Dental Rider Filing,H10I.000 Health - Dental,Rate,Closed - Approved,AMMS-126476226,126476226,https://filingaccess.serff.com/sfa/search/fili...
11417,Golden Rule Insurance Company,62286,Colorado Dental Stand Alone,H10I.000 Health - Dental,Rate,Closed - Approved,AMMS-126502262,126502262,https://filingaccess.serff.com/sfa/search/fili...
11418,Golden Rule Insurance Company,62286,Colorado Short Term,H16I.004 Short Term,Rate,Closed - Filed,AMMS-126444660,126444660,https://filingaccess.serff.com/sfa/search/fili...


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import time

# --- Load URLs ---
URLS = df['page_url']

# --- Save directly to Google Drive shared folder ---
BASE_DIR = "/Users/favasm/Library/CloudStorage/GoogleDrive-favasm@annolive.com/Shared drives/MJ/scraping/downloads"

# --- Configure Chrome ---
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,  # prevents Chrome’s built-in PDF viewer
})

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# --- Prepare new columns ---
df['attachments'] = [[] for _ in range(len(df))]

# --- Main loop ---
for idx, url in enumerate(URLS, 1):
    folder_name = url.split("=")[-1].strip()
    download_path = os.path.join(BASE_DIR, folder_name)
    os.makedirs(download_path, exist_ok=True)

    print(f"\n[{idx}/{len(URLS)}] Opening: {url}")
    print(f"Saving to: {download_path}")

    # Update Chrome's download folder for this page
    driver.execute_cdp_cmd("Page.setDownloadBehavior", {
        "behavior": "allow",
        "downloadPath": download_path
    })

    driver.get(url)

    # Wait for rows containing attachments
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.row"))
        )
    except:
        print("⚠️ No data rows found.")
        continue

    rows = driver.find_elements(By.CSS_SELECTOR, "div.row")

    for row in rows:
        try:
            # Extract form name (like "Policy")
            form_name = row.find_element(By.CSS_SELECTOR, "div.col-lg-4.summaryScheduleItemData").text.strip()

            # Find attachment link
            link = row.find_element(By.CSS_SELECTOR, "a[id*='downloadAttachment_']")

            # Remove target=_blank
            driver.execute_script("arguments[0].removeAttribute('target');", link)

            file_name = f"{form_name}.pdf"
            file_path = os.path.join(download_path, file_name)
            print(f"   ⬇️ Downloading: {file_name}")

            # Click link to download
            driver.execute_script("arguments[0].click();", link)
            time.sleep(3)  # Wait for file to start downloading

            # Append path to dataframe
            df.at[idx-1, 'attachments'].append(file_path)

        except Exception as e:
            continue

    print(f"✅ Finished downloads for {folder_name}")
    time.sleep(3)

driver.quit()
print("\n🎉 All downloads completed!")

# Save updated df with file paths
df.to_csv("downloaded_with_paths.csv", index=False)



[1/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130322935
Saving to: /Users/favasm/Library/CloudStorage/GoogleDrive-favasm@annolive.com/Shared drives/MJ/scraping/downloads/130322935
✅ Finished downloads for 130322935

[2/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130145349
Saving to: /Users/favasm/Library/CloudStorage/GoogleDrive-favasm@annolive.com/Shared drives/MJ/scraping/downloads/130145349
✅ Finished downloads for 130145349

[3/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130144986
Saving to: /Users/favasm/Library/CloudStorage/GoogleDrive-favasm@annolive.com/Shared drives/MJ/scraping/downloads/130144986
✅ Finished downloads for 130144986

[4/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130324114
Saving to: /Users/favasm/Library/CloudStorage/GoogleDrive-favasm@annolive.com/Shared drives/MJ/scraping/downloads/13032

KeyboardInterrupt: 

In [11]:
df

,Company Name,NAIC Company Code,Insurance Product Name,Sub Type Of Insurance,Filing Type,Filing Status,SERFF Tracking Number,serf_num,page_url,attachments
0,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Grp Form 2015,H11G.003 Long Term,Form,Closed - Filed,CMPL-130322935,130322935,https://filingaccess.serff.com/sfa/search/fili...,[]
1,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Large Small Grp Form 2015,H11G.003 Long Term,Form,Closed - Rejected,CMPL-130145349,130145349,https://filingaccess.serff.com/sfa/search/fili...,[]
2,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Large Grp Rate 2015,H11G.003 Long Term,Rate,Closed - Rejected,CMPL-130144986,130144986,https://filingaccess.serff.com/sfa/search/fili...,[]
3,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Large Grp Rate 2015,H11G.003 Long Term,Rate,Closed - Filed,CMPL-130324114,130324114,https://filingaccess.serff.com/sfa/search/fili...,[]
4,Security Mutual Life Insurance Company of New ...,68772,SML - LTD Small Grp Rate 2015,H11G.003 Long Term,Rate,Closed - Rejected,CMPL-130145065,130145065,https://filingaccess.serff.com/sfa/search/fili...,[]
...,...,...,...,...,...,...,...,...,...,...
11415,Golden Rule Insurance Company,62286,CO Rates,H15I.001 Health - Hospital/Surgical/Medical Ex...,Rate,Closed - Filed,AMMS-126335346,126335346,https://filingaccess.serff.com/sfa/search/fili...,[]
11416,Golden Rule Insurance Company,62286,Colorado Dental Rider Filing,H10I.000 Health - Dental,Rate,Closed - Approved,AMMS-126476226,126476226,https://filingaccess.serff.com/sfa/search/fili...,[]
11417,Golden Rule Insurance Company,62286,Colorado Dental Stand Alone,H10I.000 Health - Dental,Rate,Closed - Approved,AMMS-126502262,126502262,https://filingaccess.serff.com/sfa/search/fili...,[]
11418,Golden Rule Insurance Company,62286,Colorado Short Term,H16I.004 Short Term,Rate,Closed - Filed,AMMS-126444660,126444660,https://filingaccess.serff.com/sfa/search/fili...,[]


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import time
import shutil

# --- Load URLs ---
URLS = df['page_url']

# --- Base download directory (same folder as notebook) ---
BASE_DIR = "/Users/favasm/Library/CloudStorage/GoogleDrive-favasm@annolive.com/Shared drives/MJ/scraping/downloads"

# --- Configure Chrome ---
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,  # prevents Chrome’s built-in PDF viewer
})

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# --- Main loop ---
for idx, url in enumerate(URLS, 1):
    folder_name = url.split("=")[-1].strip()
    download_path = os.path.join(BASE_DIR, folder_name)
    os.makedirs(download_path, exist_ok=True)

    print(f"\n[{idx}/{len(URLS)}] Opening: {url}")
    print(f"Saving to: {download_path}")

    # Update Chrome's download folder for this page
    driver.execute_cdp_cmd("Page.setDownloadBehavior", {
        "behavior": "allow",
        "downloadPath": download_path
    })

    driver.get(url)

    # Wait for attachment links
    try:
        WebDriverWait(driver, 25).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[id*='downloadAttachment_']"))
        )
    except:
        print("⚠️ No attachments found or timeout.")
        continue

    links = driver.find_elements(By.CSS_SELECTOR, "a[id*='downloadAttachment_']")
    print(f"Found {len(links)} attachment(s).")

    for j, link in enumerate(links, 1):
        # Remove target="_blank" to stay in same window
        driver.execute_script("arguments[0].removeAttribute('target');", link)
        file_name = link.text.strip() or f"file_{j}.pdf"
        print(f"   ⬇️ Downloading: {file_name}")

        # Click in same tab
        driver.execute_script("arguments[0].click();", link)

        # Small wait for download to trigger
        time.sleep(3)

    # Wait a bit before next URL
    time.sleep(5)
    print(f"✅ Finished downloads for {folder_name}")

print("\n🎉 All downloads completed successfully!")
driver.quit()




[1/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130322935
Saving to: /Users/favasm/Desktop/consult/MJ/scraping/downloads/130322935
⚠️ No attachments found or timeout.

[2/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130145349
Saving to: /Users/favasm/Desktop/consult/MJ/scraping/downloads/130145349
⚠️ No attachments found or timeout.

[3/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130144986
Saving to: /Users/favasm/Desktop/consult/MJ/scraping/downloads/130144986
⚠️ No attachments found or timeout.

[4/11420] Opening: https://filingaccess.serff.com/sfa/search/filingSummary.xhtml?filingId=130324114
Saving to: /Users/favasm/Desktop/consult/MJ/scraping/downloads/130324114
Found 8 attachment(s).
   ⬇️ Downloading: SML 2015 LTD Manual Rate Tables_Revised CO 12032015.pdf
   ⬇️ Downloading: SML 2015 LTD Manual Rate Tables_Revised CO 09142015.pdf
   ⬇️ Downloading: CO S

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=141.0.7390.66)
Stacktrace:
0   chromedriver                        0x00000001012c75f0 cxxbridge1$str$ptr + 2894960
1   chromedriver                        0x00000001012bf52c cxxbridge1$str$ptr + 2861996
2   chromedriver                        0x0000000100de55ec _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74324
3   chromedriver                        0x0000000100dbf198 chromedriver + 143768
4   chromedriver                        0x0000000100e553fc _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 532580
5   chromedriver                        0x0000000100e6dfb8 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 633888
6   chromedriver                        0x0000000100e21178 _RNvCs47EqcsrPRmA_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 318944
7   chromedriver                        0